In [1]:
import numpy as np
import pandas as pd

### PARTIE 1 : Exploration de la base de données

In [11]:
df = pd.read_csv('csv_files/regularite-mensuelle-tgv-aqst.csv',sep=';')
missing_data = df.isnull().sum()
print('Donées manquantes : ', '\n', missing_data[missing_data>0])
print('Type des données : ', '\n', df.dtypes)

Donées manquantes :  
 commentaire_annulation         8154
commentaire_retards_depart     8154
commentaires_retard_arrivee    7456
dtype: int64
Type des données :  
 date                                     object
service                                  object
gare_depart                              object
gare_arrivee                             object
duree_moyenne                             int64
nb_train_prevu                            int64
nb_annulation                             int64
commentaire_annulation                  float64
nb_train_depart_retard                    int64
retard_moyen_depart                     float64
retard_moyen_tous_trains_depart         float64
commentaire_retards_depart              float64
nb_train_retard_arrivee                   int64
retard_moyen_arrivee                    float64
retard_moyen_tous_trains_arrivee        float64
commentaires_retard_arrivee              object
nb_train_retard_sup_15                    int64
retard_moyen_train

In [ ]:
print('Information : ', '\n', df.info())
print('Description statistique de chaque colonne : ', '\n', df.describe())

### PARTIE 2 : Traitement de la base de données
#### Gestion des commentaires

On observe que les seules valeurs manquantes sont pour les features commentaires : commentaire_annulation, commentaire_retards_depart
et commentaire_retard_arrivee.
Pour une première approche simplificatrice du dataset, on prend la décision de supprimier les features commentaires.

In [3]:
df_clean = (
    df.copy()
        .drop(
            ["commentaire_annulation", "commentaire_retards_depart", "commentaires_retard_arrivee"],
            axis=1,
        )
)

df_clean.columns

Index(['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne',
       'nb_train_prevu', 'nb_annulation', 'nb_train_depart_retard',
       'retard_moyen_depart', 'retard_moyen_tous_trains_depart',
       'nb_train_retard_arrivee', 'retard_moyen_arrivee',
       'retard_moyen_tous_trains_arrivee', 'nb_train_retard_sup_15',
       'retard_moyen_trains_retard_sup15', 'nb_train_retard_sup_30',
       'nb_train_retard_sup_60', 'prct_cause_externe', 'prct_cause_infra',
       'prct_cause_gestion_trafic', 'prct_cause_materiel_roulant',
       'prct_cause_gestion_gare', 'prct_cause_prise_en_charge_voyageurs'],
      dtype='object')

#### Gestion de la colonne service

In [4]:
df_clean["service"].unique()

array(['National', 'International'], dtype=object)

On observe que la colonne ne possède que deux valeurs categorielles, nous allons faire un "one-hot" encoding et creer une feature booléenne "national_service"

In [5]:
df_clean["national_service"] = np.where(df_clean["service"]=="National", 1,0)
df_clean.drop("service", axis=1, inplace=True)

#### Gestion de la date
Nous allons encoder la date en pd.Datetime64, ce format nous permettra de realiser des analyses en timeseries.


In [6]:
df_clean["date"] = pd.to_datetime(df_clean["date"])

#### Gestion des villes
La dernière étape du preprocessing est celle du traitement des colonnes gare_depart et gare_arrivee. Pour garde l'information gare de départ ou d'arrivée, on crée une colonne par ville de départ, avec le préfixe DEPART, et de même avec les villes d'arrivée, avec le préfixe ARRIVEE. On décide ainsi de garder la visibilité sur les caractéristiques de départ et d'arrivée des gares, car en effet la gare du départ ou d'arrivée peut avoir un impact sur le retard indépendemment du trajet.

In [7]:
depart = pd.get_dummies(df_clean['gare_depart'],prefix='DEPART') # Créer une colonne par ville de départ
df_clean= pd.concat([df_clean,depart], axis=1).drop('gare_depart', axis=1) ## Concaténer les colonnes et supprimer la colonne avec les données catégorilles
arrivee = pd.get_dummies(df_clean['gare_arrivee'],prefix='ARRIVEE') # Créer une colonne par ville d'arrivée'
df_clean= pd.concat([df_clean,arrivee], axis=1).drop('gare_arrivee', axis=1) 
df_clean 

,date,duree_moyenne,nb_train_prevu,nb_annulation,nb_train_depart_retard,retard_moyen_depart,retard_moyen_tous_trains_depart,nb_train_retard_arrivee,retard_moyen_arrivee,retard_moyen_tous_trains_arrivee,...,ARRIVEE_ST PIERRE DES CORPS,ARRIVEE_STRASBOURG,ARRIVEE_STUTTGART,ARRIVEE_TOULON,ARRIVEE_TOULOUSE MATABIAU,ARRIVEE_TOURCOING,ARRIVEE_TOURS,ARRIVEE_VALENCE ALIXAN TGV,ARRIVEE_VANNES,ARRIVEE_ZURICH
0,2018-01-01,141,870,5,289,11.247809,3.693179,147,28.436735,6.511118,...,False,False,False,False,False,False,False,False,False,False
1,2018-01-01,165,222,0,8,2.875000,0.095796,34,21.524020,5.696096,...,False,False,False,False,False,False,False,False,False,False
2,2018-01-01,220,248,1,37,9.501351,1.003981,26,55.692308,7.578947,...,False,False,False,False,False,False,False,False,False,False
3,2018-01-01,156,102,0,12,19.912500,1.966667,13,48.623077,6.790686,...,False,False,False,False,False,False,False,False,False,False
4,2018-01-01,61,391,2,61,7.796995,0.886889,71,12.405164,3.346487,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8149,2023-06-01,114,492,2,273,11.989927,6.742041,105,30.157302,7.689898,...,False,False,False,False,False,False,False,False,False,False
8150,2023-06-01,273,215,0,24,28.730556,3.181240,36,76.688889,14.824264,...,False,False,False,False,False,False,False,False,False,False
8151,2023-06-01,78,192,1,20,31.281667,3.276353,32,45.841146,8.650349,...,False,False,False,False,False,False,False,False,False,False
8152,2023-06-01,133,440,2,347,13.517051,10.271499,116,41.487213,12.765753,...,False,False,False,False,False,False,False,False,False,False


### Exportation de la base de donnée traitée

In [8]:
df_clean.to_csv('/csv_files/retard_train_bdd_clean.csv', index=False)